In [1]:
import pandas as pd
import string

In [2]:
nyc_path = '../clean_datasets/NYC_CLEAN.csv'
boston_path = '../clean_datasets/BOSTON_CLEAN.csv'

In [3]:
nyc_df = pd.read_csv(nyc_path)
boston_df = pd.read_csv(boston_path)

In [26]:
nyc_df.columns

Index(['Unnamed: 0', 'id', 'host_id', 'neighbourhood_group', 'neighbourhood',
       'latitude', 'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'calculated_host_listings_count',
       'availability_365', 'room_type_cat', 'neighbourhood_cat',
       'neighbourhood_group_cat'],
      dtype='object')

In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
boston_dataset = boston_df[["bathrooms","bedrooms", "beds", "bed_type", "price", "cleaning_fee", "availability_365", 
                            "number_of_reviews", "latitude", "longitude", "property_type", "room_type",
                            "neighbourhood_cleansed", "minimum_nights", "maximum_nights"]]

In [23]:
nyc_dataset = nyc_df[["price", "number_of_reviews", "latitude", "longitude", "room_type",
    "neighbourhood_group", "minimum_nights", "availability_365"]]

In [24]:
boston_dataset.isna().sum()

bathrooms                    0
bedrooms                     0
beds                         0
bed_type                     0
price                        0
cleaning_fee              1107
availability_365             0
number_of_reviews            0
latitude                     0
longitude                    0
property_type                0
room_type                    0
neighbourhood_cleansed       0
minimum_nights               0
maximum_nights               0
dtype: int64

In [8]:
boston_dataset = pd.DataFrame(boston_dataset)

In [9]:
boston_dataset["cleaning_fee"] = boston_dataset["cleaning_fee"].apply(lambda x: remove_stuff(x) if isinstance(x, str) else x).fillna(68)

In [10]:
def remove_stuff(value: str):
    word = []
    value = value.replace("$", "")
    for c in value:
        if not c.isalpha():
            word.append(c)
    try:
        new_value = float(''.join(word))
    except ValueError:
        return 0
    return float(''.join(word))

def check_availability(value: str):
    try:
        return int(value)
    except ValueError:
        return 1

In [11]:
boston_dataset['availability_30'] = boston_dataset['availability_30'].apply(lambda x: check_availability(x) if isinstance(x, str) else x).fillna(1)

In [14]:
boston_dataset.to_csv('./BOSTON_CLEANER.csv')

In [15]:
boston_dataset.head()

,bathrooms,bedrooms,beds,bed_type,price,cleaning_fee,availability_30,availability_90,number_of_reviews,latitude,longitude,property_type,room_type,neighbourhood_cleansed,minimum_nights,maximum_nights
0,1.5,2.0,3.0,Real Bed,250.0,35.0,0,0,0,42.282619,-71.133068,House,Entire home/apt,Roslindale,2,1125
1,1.0,1.0,1.0,Real Bed,65.0,10.0,26,84,36,42.286241,-71.134374,Apartment,Private room,Roslindale,2,15
2,1.0,1.0,1.0,Real Bed,65.0,68.0,19,61,41,42.292438,-71.135765,Apartment,Private room,Roslindale,3,45
3,1.0,1.0,2.0,Real Bed,75.0,50.0,6,26,1,42.281106,-71.121021,House,Private room,Roslindale,1,1125
4,1.5,1.0,2.0,Real Bed,79.0,15.0,13,59,29,42.284512,-71.136258,House,Private room,Roslindale,2,31


In [28]:
a = pd.get_dummies(boston_dataset, columns=["property_type", "room_type", "neighbourhood_cleansed", "bed_type"])

In [25]:
nyc_dataset

,price,number_of_reviews,latitude,longitude,room_type,neighbourhood_group,minimum_nights,availability_365
0,149,9,40.64749,-73.97237,Private room,Brooklyn,1,365
1,225,45,40.75362,-73.98377,Entire home/apt,Manhattan,1,355
2,150,0,40.80902,-73.94190,Private room,Manhattan,3,365
3,89,270,40.68514,-73.95976,Entire home/apt,Brooklyn,1,194
4,80,9,40.79851,-73.94399,Entire home/apt,Manhattan,10,0
...,...,...,...,...,...,...,...,...
48890,70,0,40.67853,-73.94995,Private room,Brooklyn,2,9
48891,40,0,40.70184,-73.93317,Private room,Brooklyn,4,36
48892,115,0,40.81475,-73.94867,Entire home/apt,Manhattan,10,27
48893,55,0,40.75751,-73.99112,Shared room,Manhattan,1,2


In [38]:
def calculate_labels(price):
    if price < 100:
        return 0
    elif price > 100 and price < 200:
        return 1
    elif price > 200 and price < 300:
        return 2
    elif price > 300:
        return 3
    else:
        return 3

In [39]:
a["labels"] = a["price"].apply(lambda x: calculate_labels(x))

In [40]:
a.to_csv("../clean_datasets/boston_data.csv")

In [45]:
a['cleaning_fee'] = a["cleaning_fee"].fillna(68)

In [63]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [55]:
train = a.drop(labels=['labels'], axis=1)
labels = a['labels']

In [51]:
kmeans = KMeans(n_clusters=4, random_state=100)

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(train, labels, test_size=0.3)

In [59]:
kmeans.fit(X_train)

KMeans(n_clusters=4, random_state=100)

In [60]:
train_set = kmeans.predict(X_train)

In [65]:
pca = PCA(n_components=2).fit(X_train)

In [66]:
pca_train = pca.transform(X_train)